In [224]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from matplotlib import patches
from sklearn.impute import KNNImputer
from manager import manager
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


Caricamento di Train-test e Test-set

In [225]:
trainset = pd.read_csv("./house-prices-advanced-regression-techniques/train.csv")
lenght_trainset = trainset.shape
#print(lenght_trainset)
#lenght
trainset.head(10)
testset = pd.read_csv("./house-prices-advanced-regression-techniques/test.csv")
lenght_testset = testset.shape
#print(lenght_testset)
#testset

L'idea adesso è vedere quali features nel testset non sono presenti nel trainset e viceversa, essendo la dimensione delle colonne diversa

In [226]:
#check if every feature in trainset is also present in testset and viceversa
notInTestset = []
notInTrainset = []
for x in trainset.columns:
    if not testset.columns.__contains__(x):
        notInTestset.append(x)
print(f"Columns in trainset but not in testset: {notInTestset} \n")
for x in testset.columns:
    if not trainset.columns.__contains__(x):
        notInTrainset.append(x)
print(f"Columns in testset but not in trainset: {notInTrainset} \n")
        


Columns in trainset but not in testset: ['SalePrice'] 

Columns in testset but not in trainset: [] 



In [227]:
#info of trainset
trainset_info = trainset.info()
print(trainset_info)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [228]:
#info of testset
testset_info = testset.info()
print(testset_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

eliminazione della feature id perchè non rilevante 

In [229]:
#delete feature "ID" from trainset beacause it is not relevant
if "Id" in trainset.columns:
    trainset.drop(["Id"], axis="columns", inplace=True)

#delete feature "ID" from testset and save it
if "Id" in testset.columns:
    idList = testset['Id'].tolist()
    testset.drop(['Id'], axis="columns", inplace=True)
print(trainset.shape)
    

(1460, 80)


divisione di trainset e target

In [230]:
target = trainset.iloc[:, len(trainset.columns)-1]
target.head(10)
trainset.drop(target.name, axis="columns", inplace=True)
trainset.head(10)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


eliminazione delle righe con più del 50% di valori nan

In [231]:
nanPercInARow = []
for index, row in trainset.iterrows():
    nanPercInARow = trainset.loc[[index]].isna().sum().sum() / len(trainset.columns)
    if nanPercInARow > 0.5:
        trainset.drop(index, axis="rows", inplace=True)
print(len(trainset))


1460


rimozione delle colonne con più del 95% di valori uguali.

In [232]:
#print(trainset.shape)
#print(testset.shape)
for col in trainset.columns:
    if trainset[col].unique().size == 1:
        trainset.drop(col, axis="columns", inplace=True)
    else:
        values = trainset[col].value_counts(dropna=False)
        if len(values)<5:
            #print(values)
            for value in values:
                if float(value) > float(len(trainset)) * 0.95:
                    trainset.drop(col, axis="columns", inplace=True)
                    testset.drop(col,axis="columns", inplace=True)
                    #print(col)
                    #print(value)
                    break
   
print(trainset.shape)     
print(testset.shape)   

        
            
        
for col in testset.columns:
    if testset[col].unique().size == 1:
        testset.drop(col, axis="columns", inplace=True)
    else:
        values = testset[col].value_counts(dropna=False)
        if len(values)<5:
            for value in values:
                if float(value) > float(len(testset)) * 0.95:
                    testset.drop(col, axis="columns", inplace=True)
                    trainset.drop(col,axis="columns", inplace=True)
                    break

print(trainset.shape)     
print(testset.shape)  
#print(trainset)

(1460, 75)
(1459, 75)
(1460, 71)
(1459, 71)


Parsing del dataset.
features numeriche
features categoriche.


In [233]:
#parsing trainset
trainset_num = trainset.select_dtypes(include=[np.number])
trainset_cat = trainset.select_dtypes(exclude=[np.number])



#parsing testset
testset_num = testset.select_dtypes(include=[np.number])
testset_cat = testset.select_dtypes(exclude=[np.number])
print(trainset_cat.columns)
print(trainset_num.columns)
print(trainset_cat.shape)

Index(['MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence',
       'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'Op

abbiamo 36 variabili numeriche indipendenti nel testset_num

encoding categorical features

In [234]:
#concatenazione testset_cat e trainset_cat per avere stesso encoding

lastTrainsetIndex = len(trainset_cat)-1
print(trainset_cat.shape)
print(testset_cat.shape)

df_cat =pd.concat([trainset_cat, testset_cat])

df_cat.shape

(1460, 36)
(1459, 36)


(2919, 36)

In [235]:
#encoding trainset
featureToEncode = []
for col in df_cat.columns:
    featureToEncode.append(col)
    #print(str(col)+ "\n")
    #print(trainset_cat[col].unique())
    #trainset_cat[col] = trainset_cat[col].astype['category']

le = LabelEncoder()
for col in trainset_cat.select_dtypes(include='O').columns:
    df_cat[col]=le.fit_transform(df_cat[col])
    
#encoder = OneHotEncoder(drop="first")

#df_cat_encoded = encoder.fit_transform(df_cat)
#df_cat = pd.DataFrame(df_cat_encoded.toarray(), columns=encoder.get_feature_names(), index=df_cat.index)

#split del df_cat in trainset_cat e testset_cat dopo l'encoding

trainset_cat = df_cat.iloc[:lastTrainsetIndex + 1 , :]
testset_cat = df_cat.iloc[lastTrainsetIndex + 1: , :]

#concatenazione di trainset_num e trainset_cat dopo l'encoding

trainset = pd.concat([trainset_num, trainset_cat], axis="columns")


#concatenazione di trainset_num e trainset_cat dopo l'encoding

testset = pd.concat([testset_num, testset_cat], axis="columns")


print(trainset_cat.shape)
print(testset_cat.shape)

testset.head(10)
trainset.head(10)

    

(1460, 36)
(1459, 36)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,6,5,1,1,4,4,2,4,8,4
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,6,4,1,1,4,4,2,4,8,4
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,6,4,1,1,4,4,2,4,8,4
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,6,2,5,2,4,4,2,4,8,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,6,4,1,1,4,4,2,4,8,4
5,50,85.0,14115,5,5,1993,1995,0.0,732,0,...,6,5,1,2,4,4,2,2,8,4
6,20,75.0,10084,8,5,2004,2005,186.0,1369,0,...,6,2,1,1,4,4,2,4,8,4
7,60,NaN,10382,7,6,1973,1973,240.0,859,32,...,6,4,1,1,4,4,2,4,8,4
8,50,51.0,6120,7,5,1931,1950,0.0,0,0,...,2,4,5,2,1,4,2,4,8,0
9,190,50.0,7420,5,6,1939,1950,0.0,851,0,...,6,4,1,1,2,4,2,4,8,4


gestione dei nan: utilizzo di knn

In [236]:
#nan management
trainset.isna().sum()
#use knn
imputer = KNNImputer(n_neighbors=1, weights="uniform")
trainset[:] = imputer.fit_transform(trainset)

In [237]:
#trainset_num.isna().sum()
trainset.head(10)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,6.0,5.0,1.0,1.0,4.0,4.0,2.0,4.0,8.0,4.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,6.0,4.0,1.0,1.0,4.0,4.0,2.0,4.0,8.0,4.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,6.0,4.0,1.0,1.0,4.0,4.0,2.0,4.0,8.0,4.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,6.0,2.0,5.0,2.0,4.0,4.0,2.0,4.0,8.0,0.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,6.0,4.0,1.0,1.0,4.0,4.0,2.0,4.0,8.0,4.0
5,50.0,85.0,14115.0,5.0,5.0,1993.0,1995.0,0.0,732.0,0.0,...,6.0,5.0,1.0,2.0,4.0,4.0,2.0,2.0,8.0,4.0
6,20.0,75.0,10084.0,8.0,5.0,2004.0,2005.0,186.0,1369.0,0.0,...,6.0,2.0,1.0,1.0,4.0,4.0,2.0,4.0,8.0,4.0
7,60.0,97.0,10382.0,7.0,6.0,1973.0,1973.0,240.0,859.0,32.0,...,6.0,4.0,1.0,1.0,4.0,4.0,2.0,4.0,8.0,4.0
8,50.0,51.0,6120.0,7.0,5.0,1931.0,1950.0,0.0,0.0,0.0,...,2.0,4.0,5.0,2.0,1.0,4.0,2.0,4.0,8.0,0.0
9,190.0,50.0,7420.0,5.0,6.0,1939.0,1950.0,0.0,851.0,0.0,...,6.0,4.0,1.0,1.0,2.0,4.0,2.0,4.0,8.0,4.0


correlazione

In [238]:
trainset =pd.concat([trainset, target], axis="columns")
correlation_matrix = trainset.corr()
plt.figure(figsize=(15, 16), dpi=160)
#sns.heatmap(correlation_matrix, cmap="bwr")
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "10pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '80px'),
                        ('font-size', '10pt')])
]

correlation_matrix.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '40px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Alley,LotShape,LandContour,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition,SalePrice
MSSubClass,1.00,-0.35,-0.14,0.03,-0.06,0.03,0.04,0.03,-0.07,-0.07,-0.14,-0.24,-0.25,0.31,0.05,0.07,0.00,-0.00,0.13,0.18,-0.02,0.04,-0.05,0.07,-0.04,-0.10,-0.01,-0.01,-0.01,-0.04,-0.03,0.01,-0.01,-0.01,-0.02,0.04,-0.11,0.12,-0.00,0.08,-0.01,-0.02,-0.04,0.75,0.40,-0.12,-0.09,-0.14,-0.03,-0.02,-0.00,0.06,-0.04,-0.01,-0.06,0.02,0.04,0.02,-0.10,0.05,0.01,0.00,0.06,0.13,0.03,0.04,0.04,-0.06,0.09,0.01,-0.02,-0.08
LotFrontage,-0.35,1.00,0.38,0.20,-0.04,0.09,0.05,0.16,0.22,0.08,0.09,0.35,0.41,0.09,0.03,0.38,0.09,0.03,0.19,0.06,0.26,0.32,0.24,0.05,0.25,0.31,0.10,0.12,0.04,0.06,0.06,0.14,0.01,0.02,0.02,-0.10,0.13,-0.17,-0.07,-0.18,0.07,0.00,0.02,-0.43,0.06,0.13,0.05,0.07,-0.02,-0.14,0.03,0.07,-0.13,0.02,-0.12,-0.04,-0.08,-0.05,0.07,0.06,-0.15,0.00,-0.18,-0.26,-0.21,0.02,0.00,0.09,-0.03,-0.04,0.03,0.32
LotArea,-0.14,0.38,1.00,0.11,-0.01,0.01,0.01,0.10,0.21,0.11,-0.00,0.26,0.30,0.05,0.00,0.26,0.16,0.05,0.13,0.01,0.12,0.19,0.27,-0.01,0.15,0.18,0.17,0.08,-0.02,0.02,0.04,0.08,0.04,0.00,-0.01,-0.03,0.06,-0.17,-0.15,-0.12,0.04,0.02,0.02,-0.21,-0.03,0.08,0.00,-0.00,-0.00,-0.06,0.02,-0.01,-0.07,0.01,-0.14,-0.06,-0.07,0.00,0.05,0.04,-0.06,-0.02,-0.15,-0.15,-0.12,-0.01,-0.00,0.02,0.04,0.01,0.03,0.26
OverallQual,0.03,0.20,0.11,1.00,-0.09,0.57,0.55,0.41,0.24,-0.06,0.31,0.54,0.48,0.30,-0.03,0.59,0.11,-0.04,0.55,0.27,0.10,0.43,0.40,0.54,0.60,0.56,0.24,0.31,-0.11,0.03,0.06,0.07,-0.03,0.07,-0.03,-0.16,0.10,-0.19,0.03,-0.03,0.20,0.10,0.01,0.05,0.22,0.17,0.16,0.15,0.02,-0.63,0.14,0.40,-0.61,-0.00,-0.27,-0.01,0.05,-0.43,0.27,0.25,-0.56,0.16,-0.43,-0.43,-0.56,-0.01,0.02,0.23,0.15,-0.06,0.19,0.79
OverallCond,-0.06,-0.04,-0.01,-0.09,1.00,-0.38,0.07,-0.13,-0.05,0.04,-0.14,-0.17,-0.14,0.03,0.03,-0.08,-0.05,0.12,-0.19,-0.06,0.01,-0.06,-0.02,-0.30,-0.19,-0.15,-0.00,-0.03,0.07,0.03,0.05,-0.00,0.07,-0.00,0.04,0.19,-0.10,0.01,-0.05,-0.03,-0.03,-0.03,0.01,-0.16,-0.03,-0.01,-0.04,-0.03,-0.00,0.17,-0.23,-0.29,0.21,-0.02,0.06,-0.09,-0.11,0.05,0.12,0.06,0.07,0.09,0.04,0.14,0.17,-0.09,-0.03,-0.06,-0.16,0.10,0.02,-0.08
YearBuilt,0.03,0.09,0.01,0.57,-0.38,1.00,0.59,0.31,0.25,-0.05,0.15,0.39,0.28,0.01,-0.18,0.20,0.19,-0.04,0.47,0.24,-0.07,0.10,0.15,0.80,0.54,0.48,0.22,0.19,-0.39,0.03,-0.05,0.00,-0.03,0.01,-0.01,-0.31,0.30,-0.23,0.16,0.01,0.10,0.16,0.03,0.22,0.27,0.05,0.10,0.08,-0.04,-0.49,0.27,0.63,-0.53,0.13,-0.31,-0.15,0.04,-0.40,0.38,0.34,-0.36,0.15,-0.17,-0.54,-0.60,0.20,0.13,0.43,0.20,-0.05,0.20,0.52
YearRemodAdd,0.04,0.05,0.01,0.55,0.07,0.59,1.00,0.18,0.13,-0.07,0.18,0.29,0.24,0.14,-0.06,0.29,0.12,-0.01,0.44,0.18,-0.04,0.19,0.11,0.61,0.42,0.37,0.21,0.23,-0.19,0.05,-0.04,0.01,-0.01,0.02,0.04,-0.17,0.07,-0.15,0.09,-0.01,0.10,0.14,0.01,0.10,0.21,0.03,0.24,0.22,0.07,-0.48,0.08,0.44,-0.48,-0.01,-0.23,-0.03,0.05,-0.50,0.30,0.35,-0.45,0.08,-0.17,-0.31,-0.44,0.08,0.09,0.17,0.14,0.02,0.22,0.51
MasVnrArea,0.03,0.16,0.10,0.41,-0.13,0.31,0.18,1.00,0.27,-0.07,0.11,0.36,0.34,0.17,-0.07,0.39,0.09,0.03,0.27,0.20,0.10,0.28,0.25,0.26,0.36,0.37,0.16,0.13,-0.11,0.02,0.06,0.01,-0.03,-0.00,-0.01,-0.07,0.07,-0.11,0.05,-0.03,0.12,0.02,0.04,0.05,0.17,0.25,0.03,0.00,-0.33,-0.36,0.07,0.18,-0.32,0.05,-0.13,-0.08,0.04,-0.14,0.13,0.11,-0.26,0.09,-0.24,-0.22,-0.28,0.06,0.01,0.14,0.08,-0.06,0.09,0.48
BsmtFinSF1,-0.07,0.22,0.21,0.24,-0.05,0

<Figure size 2400x2560 with 0 Axes>

analisi della correlazione con "SalePrice"
• se |corr|>0.80 strong correlation
• se 0.5<|corr|<0.80 moderate correlation
• se 0.20<|corr|<0.5 weak correlation
• se |corr|<0.2 very weak or null correlation

In [239]:
target_correlation = trainset.corr()[target.name][:-1]
target_correlation

MSSubClass      -0.084284
LotFrontage      0.316983
LotArea          0.263843
OverallQual      0.790982
OverallCond     -0.077856
                   ...   
GarageCond       0.025149
PavedDrive       0.231357
Fence            0.140640
SaleType        -0.054911
SaleCondition    0.213092
Name: SalePrice, Length: 71, dtype: float64

In [240]:
#find features with a very weak correlations 
very_weak_corr = target_correlation[abs(target_correlation)<0.25].sort_values(ascending=False)
#print something
very_weak_corr_names = target_correlation[abs(target_correlation)<0.25].index.tolist()
#print(very_weak_corr_names)

#find features with a weak correlations
weak_corr = target_correlation[(abs(target_correlation)>=0.25) & (abs(target_correlation)<0.50)].sort_values(ascending=False)
#print something
weak_corr_list = target_correlation[(abs(target_correlation)>=0.25) & (abs(target_correlation)<0.50)].index.tolist()

#find features with a moderate correlations
moderate_corr = target_correlation[(abs(target_correlation)>=0.50) & (abs(target_correlation)<0.75)].sort_values(ascending=False)
moderate_corr_list = target_correlation[(abs(target_correlation)>=0.50) & (abs(target_correlation)<0.75)].index.tolist()
#print(moderate_corr)

#find features with a strong correlations
strong_corr = target_correlation[(abs(target_correlation)>=0.75)].sort_values(ascending=False)
strong_corr_list = target_correlation[(abs(target_correlation)>=0.75)].index.tolist()
#print(strong_corr)



In [243]:
#trainset and testset with only correlated feature
trainset = trainset.loc[:, strong_corr_list + moderate_corr_list + weak_corr_list + [target.name]]
testset = testset.loc[:, strong_corr_list + moderate_corr_list + weak_corr_list]


,OverallQual,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,GarageCars,GarageArea,...,GarageYrBlt,WoodDeckSF,OpenPorchSF,LotShape,Foundation,BsmtExposure,HeatingQC,CentralAir,FireplaceQu,GarageType
0,5,1961,1961,882.0,896,896,1,5,1.0,730.0,...,1961.0,140,0,3,1,3,4,1,5,1
1,6,1958,1958,1329.0,1329,1329,1,6,1.0,312.0,...,1958.0,393,36,0,1,3,4,1,5,1
2,5,1997,1998,928.0,928,1629,2,6,2.0,482.0,...,1997.0,212,34,0,2,3,2,1,4,1
3,6,1998,1998,926.0,926,1604,2,7,2.0,470.0,...,1998.0,360,36,0,2,3,0,1,2,1
4,8,1992,1992,1280.0,1280,1280,2,5,2.0,506.0,...,1992.0,0,82,0,2,3,0,1,5,1
5,6,1993,1994,763.0,763,1655,2,7,2.0,440.0,...,1993.0,157,84,0,2,3,2,1,4,1
6,6,1992,2007,1168.0,1187,1187,2,6,2.0,420.0,...,1992.0,483,21,0,2,3,0,1,5,1
7,6,1998,1998,789.0,789,1465,2,7,2.0,393.0,...,1998.0,0,75,0,2,3,2,1,2,1
8,7,1990,1990,1300.0,1341,1341,1,5,2.0,506.0,...,1990.0,192,0,3,2,1,2,1,3,1
9,4,1970,1970,882.0,882,882,1,4,2.0,525.0,...,1970.0,240,0,3,1,3,4,1,5,1
